# Query WikiData

In [1]:
import requests
import pandas as pd
from collections import OrderedDict

In [2]:
url = 'https://query.wikidata.org/sparql'

## Building the Query - Essentials for each query

`?event wdt:P31/wdt:P279* wd:Q198.`
Event is an instance of (P31) or subclass of (P279) war (Q198).

`?event wdt:P710 ?participants.`
Event has participants (P710) Participant.

`?event wdt:P580 ?startDate.`
Event has start time (P580) StartDate.

`FILTER(YEAR(?startDate) >= 1946).`
Event has start date after 1946

## Wars & their participants

In [3]:
query1 = """
SELECT DISTINCT
  ?event ?eventLabel ?participant ?participantLabel
WHERE {
  ?event wdt:P31/wdt:P279* wd:Q198.
  OPTIONAL { ?event wdt:P710 ?participant }
  ?event wdt:P580 ?startDate
  FILTER(YEAR(?startDate) >= 1946).
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
"""

In [4]:
r1 = requests.get(url, params = {'format': 'json', 'query': query1})
data1 = r1.json()

wars_pars = []
for item in data1['results']['bindings']:
    wars_pars.append(OrderedDict({
        label : item[label]['value'] if label in item else None
        for label in ['event', 'eventLabel', 'participant', 'participantLabel']}))

In [5]:
df1 = pd.DataFrame(wars_pars)
df1['warID'] = df1['event'].str.split('/').str[-1]
df1['participantID'] = df1['participant'].str.split('/').str[-1]

In [6]:
df1

,event,eventLabel,participant,participantLabel,warID,participantID
0,http://www.wikidata.org/entity/Q190758,Darfur conflict,http://www.wikidata.org/entity/Q384535,African Union – United Nations Hybrid Operatio...,Q190758,Q384535
1,http://www.wikidata.org/entity/Q322607,War of Attrition,http://www.wikidata.org/entity/Q801,Israel,Q322607,Q801
2,http://www.wikidata.org/entity/Q322607,War of Attrition,http://www.wikidata.org/entity/Q241,Cuba,Q322607,Q241
3,http://www.wikidata.org/entity/Q322607,War of Attrition,http://www.wikidata.org/entity/Q810,Jordan,Q322607,Q810
4,http://www.wikidata.org/entity/Q303801,Lord's Resistance Army insurgency,http://www.wikidata.org/entity/Q1036,Uganda,Q303801,Q1036
...,...,...,...,...,...,...
847,http://www.wikidata.org/entity/Q18920712,War in Afghanistan (2015–),http://www.wikidata.org/entity/Q42418,Taliban,Q18920712,Q42418
848,http://www.wikidata.org/entity/Q18920712,War in Afghanistan (2015–),http://www.wikidata.org/entity/Q145,United Kingdom,Q18920712,Q145
849,http://www.wikidata.org/entity/Q11154636,Eelam War III,http://www.wikidata.org/entity/Q854,Sri Lanka,Q11154636,Q854
850,http://www.wikidata.org/entity/Q18920712,War in Afghanistan (2015–),http://www.wikidata.org/entity/Q30,United States of America,Q18920712,Q30


In [7]:
df1['warID'].nunique()

195

In [8]:
df1['participantID'].nunique()

495

In [9]:
df1.to_csv("Data/wars_participants.csv", index=False, encoding='utf-8')

## Wars

In [10]:
query2 = """
SELECT DISTINCT
  ?event ?eventLabel ?eventDescription ?startDate ?endDate ?location ?casualties ?prevWar ?nextWar ?largerConflict ?largerConflictLabel ?largerConflictDescription
WHERE {
  ?event wdt:P31/wdt:P279* wd:Q198.
  ?event wdt:P580 ?startDate.
  OPTIONAL { ?event wdt:P582 ?endDate. }
  OPTIONAL { ?event wdt:P276 ?location. }
  OPTIONAL { ?event wdt:P155 ?prevWar. }
  OPTIONAL { ?event wdt:P156 ?nextWar. }
  OPTIONAL { ?event wdt:P361 ?largerConflict. }
  OPTIONAL { ?event wdt:P1120 ?casualties }
  FILTER(YEAR(?startDate) >= 1946).
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
"""

In [11]:
r2 = requests.get(url, params = {'format': 'json', 'query': query2})
data2 = r2.json()

wars = []
for item in data2['results']['bindings']:
    wars.append(OrderedDict({
        label : item[label]['value'] if label in item else None
        for label in ['event', 'eventLabel', 'eventDescription', 'startDate', 'endDate', 'location', 
                      'casualties', 'prevWar', 'nextWar', 'largerConflict', 'largerConflictLabel', 'largerConflictDescription']}))

In [12]:
df2 = pd.DataFrame(wars)
df2['warID'] = df2['event'].str.split('/').str[-1]
df2['locationID'] = df2['location'].str.split('/').str[-1]
df2['prevWarID'] = df2['prevWar'].str.split('/').str[-1]
df2['nextWarID'] = df2['nextWar'].str.split('/').str[-1]
df2['largerConflictID'] = df2['largerConflict'].str.split('/').str[-1]

In [13]:
df2

,event,eventLabel,eventDescription,startDate,endDate,location,casualties,prevWar,nextWar,largerConflict,largerConflictLabel,largerConflictDescription,warID,locationID,prevWarID,nextWarID,largerConflictID
0,http://www.wikidata.org/entity/Q82664,Iran–Iraq War,war between Iran and Iraq from September 1980 ...,1980-09-22T00:00:00Z,1988-08-20T00:00:00Z,http://www.wikidata.org/entity/Q34675,None,None,None,http://www.wikidata.org/entity/Q13408125,Gulf War,Wikimedia disambiguation page,Q82664,Q34675,None,None,Q13408125
1,http://www.wikidata.org/entity/Q8663,Korean War,1950–1953 war between North Korea and South Korea,1950-06-25T00:00:00Z,1953-07-27T00:00:00Z,http://www.wikidata.org/entity/Q27092,None,None,None,http://www.wikidata.org/entity/Q8683,Cold War,state of geopolitical tension after World War ...,Q8663,Q27092,None,None,Q8683
2,http://www.wikidata.org/entity/Q185729,War on Terror,international military campaign that started a...,2001-09-11T00:00:00Z,None,None,None,None,None,None,None,None,Q185729,None,None,None,None
3,http://www.wikidata.org/entity/Q49101,Suez Crisis,diplomatic and military confrontation in late ...,1956-10-29T00:00:00Z,1956-11-07T00:00:00Z,http://www.wikidata.org/entity/Q39760,None,None,None,http://www.wikidata.org/entity/Q8669,Arab–Israeli conflict,geopolitical conflict in the Middle East and N...,Q49101,Q39760,None,None,Q8669
4,http://www.wikidata.org/entity/Q48249,Falklands War,1982 war between Argentina and the United Kingdom,1982-04-02T00:00:00Z,1982-06-14T00:00:00Z,http://www.wikidata.org/entity/Q9648,891,None,None,http://www.wikidata.org/entity/Q4379537,Falkland Islands sovereignty dispute,dispute between Argentina and the United Kingdom,Q48249,Q9648,None,None,Q4379537
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309,http://www.wikidata.org/entity/Q2354629,Malayan Emergency,guerrilla war from 1948 to 1960,1948-06-16T00:00:00Z,1960-07-12T00:00:00Z,http://www.wikidata.org/entity/Q18758,None,None,http://www.wikidata.org/entity/Q3446062,http://www.wikidata.org/entity/Q8683,Cold War,state of geopolitical tension after World War ...,Q2354629,Q18758,None,Q3446062,Q8683
310,http://www.wikidata.org/entity/Q42306428,First Burundi War,Civil War for capital in 1972,1972-01-01T00:00:00Z,1972-01-01T00:00:00Z,http://www.wikidata.org/entity/Q967,None,None,None,None,None,None,Q42306428,Q967,None,None,None
311,http://www.wikidata.org/entity/Q42306431,Second Guatemala War,Civil War for Capital between 1970 and 1971,1970-01-01T00:00:00Z,1971-01-01T00:00:00Z,http://www.wikidata.org/entity/Q774,None,None,None,http://www.wikidata.org/entity/Q1282732,Guatemalan Civil War,1960-1996 civil war in Guatemala,Q42306431,Q774,None,None,Q1282732
312,http://www.wikidata.org/entity/Q42410655,Second Liberia War,Civil War for Capital between 1992 to 1995,1992-01-01T00:00:00Z,1995-01-01T00:00:00Z,http://www.wikidata.org/entity/Q1014,None,None,None,None,None,None,Q42410655,Q1014,None,None,None


In [14]:
df2.to_csv("Data/wars.csv", index=False, encoding='utf-8')

get location details

In [15]:
query2l = """
SELECT DISTINCT
  ?location ?locationLabel ?locationDescription ?locationCountry ?locationCountryLabel
WHERE {
  ?event wdt:P31/wdt:P279* wd:Q198.
  ?event wdt:P580 ?startDate.
  ?event wdt:P276 ?location.
  OPTIONAL { ?location wdt:P17 ?locationCountry. }
  FILTER(YEAR(?startDate) >= 1946).
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
"""

In [16]:
r2l = requests.get(url, params = {'format': 'json', 'query': query2l})
data2l = r2l.json()

locations = []
for item in data2l['results']['bindings']:
    locations.append(OrderedDict({
        label : item[label]['value'] if label in item else None
        for label in ['location', 'locationLabel', 'locationDescription', 'locationCountry', 'locationCountryLabel']}))

In [17]:
df2l = pd.DataFrame(locations)
df2l['locationID'] = df2l['location'].str.split('/').str[-1]
df2l['locationCountryID'] = df2l['locationCountry'].str.split('/').str[-1]

In [18]:
df2l.to_csv("Data/war_locations.csv", index=False, encoding='utf-8')

## Participants

sovereign states

In [19]:
query3 = """
SELECT DISTINCT
  ?participant ?participantLabel ?participantDescription ?countryCode ?inception ?dissolved ?government ?governmentLabel ?governmentDescription
WHERE {
  ?event wdt:P31/wdt:P279* wd:Q198.
  ?event wdt:P710 ?participant.
  ?participant wdt:P31 ?participantType.
  OPTIONAL { ?participant wdt:P298 ?countryCode }.
  ?participant wdt:P31/wdt:P279* wd:Q3624078.
  OPTIONAL { ?participant wdt:P571 ?inception. }
  OPTIONAL { ?participant wdt:P576 ?dissolved. }
  OPTIONAL { ?participant wdt:P122 ?government. }
  ?event wdt:P580 ?startDate.
  OPTIONAL { ?event wdt:P582 ?endDate }.
  FILTER(YEAR(?startDate) >= 1946).
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
"""

In [20]:
r3 = requests.get(url, params = {'format': 'json', 'query': query3})
data3 = r3.json()

states = []
for item in data3['results']['bindings']:
    states.append(OrderedDict({
        label : item[label]['value'] if label in item else None
        for label in ['participant', 'participantLabel', 'participantDescription', 'countryCode', 'inception', 'dissolved', 
                      'government', 'governmentLabel', 'governmentDescription']}))

In [21]:
df3 = pd.DataFrame(states)
df3['participantID'] = df3['participant'].str.split('/').str[-1]
df3['govtID'] = df3['government'].str.split('/').str[-1]

In [22]:
df3

,participant,participantLabel,participantDescription,countryCode,inception,dissolved,government,governmentLabel,governmentDescription,participantID,govtID
0,http://www.wikidata.org/entity/Q114,Kenya,sovereign state in East Africa,KEN,1963-01-01T00:00:00Z,None,None,None,None,Q114,None
1,http://www.wikidata.org/entity/Q142,France,sovereign state with mainland in Europe and se...,FRA,None,None,http://www.wikidata.org/entity/Q49890,semi-presidential system,system of government,Q142,Q49890
2,http://www.wikidata.org/entity/Q36,Poland,sovereign state in Central Europe,POL,1918-11-11T00:00:00Z,None,http://www.wikidata.org/entity/Q166747,parliamentary system,form of government,Q36,Q166747
3,http://www.wikidata.org/entity/Q142,France,sovereign state with mainland in Europe and se...,FRA,None,None,http://www.wikidata.org/entity/Q188961,decentralization,process of redistributing or dispersing functi...,Q142,Q188961
4,http://www.wikidata.org/entity/Q115,Ethiopia,Country in East Africa,ETH,1941-05-05T00:00:00Z,None,http://www.wikidata.org/entity/Q512187,federal republic,federation of states or territories with a rep...,Q115,Q512187
...,...,...,...,...,...,...,...,...,...,...,...
156,http://www.wikidata.org/entity/Q224,Croatia,sovereign state in Central Europe,HRV,1991-06-25T00:00:00Z,None,http://www.wikidata.org/entity/Q7270,republic,form of government where people elect leaders ...,Q224,Q7270
157,http://www.wikidata.org/entity/Q419,Peru,sovereign state in South America,PER,1821-07-28T00:00:00Z,None,http://www.wikidata.org/entity/Q7270,republic,form of government where people elect leaders ...,Q419,Q7270
158,http://www.wikidata.org/entity/Q971,Republic of the Congo,sovereign state in Central Africa,COG,1960-01-01T00:00:00Z,None,None,None,None,Q971,None
159,http://www.wikidata.org/entity/Q774,Guatemala,sovereign state in Central America,GTM,1821-01-01T00:00:00Z,None,None,None,None,Q774,None


In [23]:
df3[df3['participantLabel']=='Egypt']

,participant,participantLabel,participantDescription,countryCode,inception,dissolved,government,governmentLabel,governmentDescription,participantID,govtID
66,http://www.wikidata.org/entity/Q79,Egypt,sovereign state in Africa and Asia,EGY,1922-02-28T00:00:00Z,None,http://www.wikidata.org/entity/Q7270,republic,form of government where people elect leaders ...,Q79,Q7270
67,http://www.wikidata.org/entity/Q79,Egypt,sovereign state in Africa and Asia,EGY,-3149-01-01T00:00:00Z,None,http://www.wikidata.org/entity/Q7270,republic,form of government where people elect leaders ...,Q79,Q7270


In [24]:
df3.to_csv("Data/participants_states.csv", index=False, encoding='utf-8')

armed organizations

In [25]:
query4 = """
SELECT DISTINCT
  ?participant ?participantLabel ?participantDescription ?inception ?dissolved ?type ?typeLabel ?typeDescription
WHERE {
  ?event wdt:P31/wdt:P279* wd:Q198.
  ?event wdt:P710 ?participant.
  ?participant wdt:P31 ?participantType.
  ?participant wdt:P31/wdt:P279* wd:Q17149090.
  OPTIONAL { ?participant wdt:P571 ?inception. }
  OPTIONAL { ?participant wdt:P576 ?dissolved. }
  ?participant wdt:P31 ?type.
  ?event wdt:P580 ?startDate.
  OPTIONAL { ?event wdt:P582 ?endDate }.
  FILTER(YEAR(?startDate) >= 1946).
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
"""

In [26]:
r4 = requests.get(url, params = {'format': 'json', 'query': query4})
data4 = r4.json()

orgs = []
for item in data4['results']['bindings']:
    orgs.append(OrderedDict({
        label : item[label]['value'] if label in item else None
        for label in ['participant', 'participantLabel', 'participantDescription', 'inception', 'dissolved', 
                      'type', 'typeLabel', 'typeDescription']}))

In [27]:
df4 = pd.DataFrame(orgs)
df4['participantID'] = df4['participant'].str.split('/').str[-1]
df4['typeID'] = df4['type'].str.split('/').str[-1]

In [28]:
df4

,participant,participantLabel,participantDescription,inception,dissolved,type,typeLabel,typeDescription,participantID,typeID
0,http://www.wikidata.org/entity/Q80312,Liberation Tigers of Tamil Eelam,militant organization in Sri Lanka,1976-01-01T00:00:00Z,2009-01-01T00:00:00Z,http://www.wikidata.org/entity/Q7278,political party,organization that seeks to influence governmen...,Q80312,Q7278
1,http://www.wikidata.org/entity/Q80312,Liberation Tigers of Tamil Eelam,militant organization in Sri Lanka,1976-01-01T00:00:00Z,2009-01-01T00:00:00Z,http://www.wikidata.org/entity/Q20642011,guerrilla movement,military organization practicing irregular war...,Q80312,Q20642011
2,http://www.wikidata.org/entity/Q58967,Israel Defense Forces,combined military forces of Israel,1948-05-26T00:00:00Z,None,http://www.wikidata.org/entity/Q772547,armed forces,collective organization of a country's armed m...,Q58967,Q772547
3,http://www.wikidata.org/entity/Q160805,United Nations Interim Force in Lebanon,None,1978-01-01T00:00:00Z,None,http://www.wikidata.org/entity/Q1456930,peacekeeping force,military unit of one or more uninvolved countr...,Q160805,Q1456930
4,http://www.wikidata.org/entity/Q7184,NATO,intergovernmental military alliance of Western...,1949-04-04T00:00:00Z,None,http://www.wikidata.org/entity/Q245065,intergovernmental organization,organization composed primarily of sovereign s...,Q7184,Q245065
...,...,...,...,...,...,...,...,...,...,...
110,http://www.wikidata.org/entity/Q1674170,Turkistan Islamic Party,None,1997-01-01T00:00:00Z,None,http://www.wikidata.org/entity/Q17127659,terrorist organization,group designated as terrorist by an authority,Q1674170,Q17127659
111,http://www.wikidata.org/entity/Q1967703,National Resistance Army,military wing of the National Resistance Movem...,1981-06-01T00:00:00Z,1995-01-01T00:00:00Z,http://www.wikidata.org/entity/Q17149090,armed organization,organization which achieves or tries to achiev...,Q1967703,Q17149090
112,http://www.wikidata.org/entity/Q935288,Libyan armed forces,combined military forces of Libya,1977-01-01T00:00:00Z,None,http://www.wikidata.org/entity/Q772547,armed forces,collective organization of a country's armed m...,Q935288,Q772547
113,http://www.wikidata.org/entity/Q1639447,Pakistan Air Force,air warfare branch of Pakistan's armed forces,1947-08-14T00:00:00Z,None,http://www.wikidata.org/entity/Q61883,air force,military branch of service primarily concerned...,Q1639447,Q61883


In [29]:
df4.to_csv("Data/participants_armedorgs.csv", index=False, encoding='utf-8')